In [4]:
# Célula 1 - Importações (igual ao anterior)
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset


# Montar o Drive (faça isso só uma vez por sessão)
from google.colab import drive
drive.mount('/content/drive')

# Caminho para o arquivo CSV no Drive
caminho_csv = '/content/drive/MyDrive/Dataset/fechamento_acoes_ultimo_ano.csv'

# Carregar o CSV
df = pd.read_csv(caminho_csv, index_col=0)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Célula 2 - Carregar e preparar dados (igual)
data = df.values

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

def criar_sequencias(data, passos=30):
    X, y = [], []
    for i in range(len(data) - passos):
        X.append(data[i:i+passos])
        y.append(data[i+passos])
    return np.array(X), np.array(y)

passos = 30
X, y = criar_sequencias(data_scaled, passos)

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

dataset = TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [6]:
# Célula 3 - Definição do modelo GRU
class GRUModel(nn.Module):
    def __init__(self, input_size=5, hidden_size=50, num_layers=1):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, input_size)
    def forward(self, x):
        out, _ = self.gru(x)
        out = out[:, -1, :]
        out = self.linear(out)
        return out

model = GRUModel()


In [7]:
# Célula 4 - Perda e otimizador
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [8]:
# Célula 5 - Treinamento
def treinar_modelo(model, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * X_batch.size(0)
        avg_loss = total_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.6f}")


In [9]:
# Célula 6 - Avaliação
def avaliar_modelo(model):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            output = model(X_batch)
            loss = criterion(output, y_batch)
            total_loss += loss.item() * X_batch.size(0)
    avg_loss = total_loss / len(test_loader.dataset)
    print(f"Loss no teste: {avg_loss:.6f}")
    return avg_loss


In [10]:
# Célula 7 - Treinar e avaliar
treinar_modelo(model, optimizer, epochs=10)
avaliar_modelo(model)


Epoch 1/10 - Loss: 0.324739
Epoch 2/10 - Loss: 0.227839
Epoch 3/10 - Loss: 0.131802
Epoch 4/10 - Loss: 0.055939
Epoch 5/10 - Loss: 0.039241
Epoch 6/10 - Loss: 0.033773
Epoch 7/10 - Loss: 0.030036
Epoch 8/10 - Loss: 0.030186
Epoch 9/10 - Loss: 0.027349
Epoch 10/10 - Loss: 0.026366
Loss no teste: 0.027301


0.027300858023491772